# 🔌 Conexión a la base de datos MySQL

Nos conectamos al esquema `world` previamente cargado en nuestro servidor local de MySQL. Esta conexión permitirá ejecutar consultas directamente desde Python, integrando la lógica SQL con el análisis y la visualización en Jupyter.

In [23]:
# --------------------------------------------------------------------------------
# CONEXIÓN A MYSQL DESDE PYTHON
# --------------------------------------------------------------------------------

import os
import mysql.connector
from dotenv import load_dotenv

load_dotenv()

# ✅ Parámetros de conexión definidos de forma explícita
config = {
    "host": os.environ["HOST"],
    "user": os.environ["DBUSER"],
    "password": os.environ["SQL_KEY"],
    "database": os.environ["DATABASE"]
}

# ✅ Establecemos la conexión usando el diccionario de configuración
try:
    conn = mysql.connector.connect(**config)
    cursor = conn.cursor()
    print("✅ Conexión exitosa a la base de datos 'world'")
except mysql.connector.Error as err:
    print(f"❌ Error al conectar a la base de datos: {err}")


✅ Conexión exitosa a la base de datos 'world'


## Procesamiento y limpieza de datos (Data Wrangling)

Antes de realizar análisis y visualizaciones, es fundamental garantizar la calidad de los datos. En esta sección se lleva a cabo la inspección, tipado, detección de valores nulos y verificación de la integridad de la tabla `country`, base para múltiples consultas posteriores.

### 1. Revisión del esquema de la tabla `country, city y countrylanguage`

Comprobamos la estructura de la tabla para conocer los nombres de columnas, tipos y claves, lo que nos ayudará a aplicar transformaciones precisas en pasos posteriores.

In [27]:
# Librerías esenciales para análisis de datos
import pandas as pd  # Manipulación y análisis de datos
import numpy as np   # Operaciones numéricas

# Consulta SQL para ver la estructura de la tabla country
query_schema = "DESCRIBE country"
pd.read_sql(query_schema, conn)

/var/folders/jt/ckhxry010gnctm46whg80gm80000gn/T/ipykernel_7231/4291458228.py:7: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  pd.read_sql(query_schema, conn)


,Field,Type,Null,Key,Default,Extra
0,Code,char(3),NO,PRI,,
1,Name,char(52),NO,,,
2,Continent,"enum('Asia','Europe','North America','Africa',...",NO,,Asia,
3,Region,char(26),NO,,,
4,SurfaceArea,"decimal(10,2)",NO,,0.00,
5,IndepYear,smallint,YES,,None,
6,Population,int,NO,,0,
7,LifeExpectancy,"decimal(3,1)",YES,,None,
8,GNP,"decimal(10,2)",YES,,None,
9,GNPOld,"decimal(10,2)",YES,,None,


In [28]:
# Consulta SQL para ver la estructura de la tabla city
query_schema = "DESCRIBE city"
pd.read_sql(query_schema, conn)

/var/folders/jt/ckhxry010gnctm46whg80gm80000gn/T/ipykernel_7231/29141934.py:3: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  pd.read_sql(query_schema, conn)


,Field,Type,Null,Key,Default,Extra
0,ID,int,NO,PRI,None,auto_increment
1,Name,char(35),NO,,,
2,CountryCode,char(3),NO,MUL,,
3,District,char(20),NO,,,
4,Population,int,NO,,0,


In [29]:
# Consulta SQL para ver la estructura de la tabla countrylanguage
query_schema = "DESCRIBE countrylanguage"
pd.read_sql(query_schema, conn)

/var/folders/jt/ckhxry010gnctm46whg80gm80000gn/T/ipykernel_7231/2293491678.py:3: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  pd.read_sql(query_schema, conn)


,Field,Type,Null,Key,Default,Extra
0,CountryCode,char(3),NO,PRI,,
1,Language,char(30),NO,PRI,,
2,IsOfficial,"enum('T','F')",NO,,F,
3,Percentage,"decimal(4,1)",NO,,0.0,


### 2. Limpieza y Preparación de Datos

Antes de iniciar el análisis exploratorio, es imprescindible asegurar la calidad de los datos. A continuación, se realiza una revisión estructural y limpieza básica de las tablas `country`, `city` y `countrylanguage`.

#### 2.1. Revisión inicial de las dimensiones y estructura

Se explora el tamaño, las primeras filas y los tipos de datos de cada tabla para tener una primera aproximación de su calidad y estructura.

In [44]:
# Dimensiones
print("Country:", df_country.shape)
print("City:", df_city.shape)
print("CountryLanguage:", df_lang.shape)

Country: (239, 15)
City: (4079, 5)
CountryLanguage: (984, 4)


In [42]:
# Primeras filas
display(df_country.head(3))
display(df_city.head(3))
display(df_lang.head(3))

,Code,Name,Continent,Region,SurfaceArea,IndepYear,Population,LifeExpectancy,GNP,GNPOld,LocalName,GovernmentForm,HeadOfState,Capital,Code2
0,ABW,Aruba,North America,Caribbean,193.0,<NA>,103000,78.4,828.0,793.0,Aruba,Nonmetropolitan Territory of The Netherlands,Beatrix,129.0,AW
1,AFG,Afghanistan,Asia,Southern and Central Asia,652090.0,1919,22720000,45.9,5976.0,NaN,Afganistan/Afqanestan,Islamic Emirate,Mohammad Omar,1.0,AF
2,AGO,Angola,Africa,Central Africa,1246700.0,1975,12878000,38.3,6648.0,7984.0,Angola,Republic,José Eduardo dos Santos,56.0,AO


,ID,Name,CountryCode,District,Population
0,1,Kabul,AFG,Kabol,1780000
1,2,Qandahar,AFG,Qandahar,237500
2,3,Herat,AFG,Herat,186800


,CountryCode,Language,IsOfficial,Percentage
0,ABW,Dutch,T,5.3
1,ABW,English,F,9.5
2,ABW,Papiamento,F,76.7


In [43]:
# Tipos de datos
df_country.info()
df_city.info()
df_lang.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 239 entries, 0 to 238
Data columns (total 15 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   Code            239 non-null    object 
 1   Name            239 non-null    object 
 2   Continent       239 non-null    object 
 3   Region          239 non-null    object 
 4   SurfaceArea     239 non-null    float64
 5   IndepYear       192 non-null    Int64  
 6   Population      239 non-null    int64  
 7   LifeExpectancy  222 non-null    float64
 8   GNP             239 non-null    float64
 9   GNPOld          178 non-null    float64
 10  LocalName       239 non-null    object 
 11  GovernmentForm  239 non-null    object 
 12  HeadOfState     238 non-null    object 
 13  Capital         232 non-null    float64
 14  Code2           239 non-null    object 
dtypes: Int64(1), float64(5), int64(1), object(8)
memory usage: 28.4+ KB
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4079 en

#### 2.2. Conversión de tipos de datos

Algunas columnas están en formato decimal o categórico, por lo que se tipifican correctamente:
- `LifeExpectancy`, `GNP`, `GNPOld`, `Percentage` → float
- `IndepYear` → entero tolerante a nulos (`Int64`)

In [39]:
# Conversión a float
df_country["LifeExpectancy"] = pd.to_numeric(df_country["LifeExpectancy"], errors="coerce")
df_country["GNP"] = pd.to_numeric(df_country["GNP"], errors="coerce")
df_country["GNPOld"] = pd.to_numeric(df_country["GNPOld"], errors="coerce")
df_lang["Percentage"] = pd.to_numeric(df_lang["Percentage"], errors="coerce")

# Conversión de año de independencia
df_country["IndepYear"] = df_country["IndepYear"].astype("Int64")
display(df_country.dtypes)

Code               object
Name               object
Continent          object
Region             object
SurfaceArea       float64
IndepYear           Int64
Population          int64
LifeExpectancy    float64
GNP               float64
GNPOld            float64
LocalName          object
GovernmentForm     object
HeadOfState        object
Capital           float64
Code2              object
dtype: object

#### 2.3. Detección de valores nulos y duplicados

Es esencial revisar la presencia de datos faltantes y eliminar duplicados si los hubiera.

In [45]:
# Porcentaje de nulos por columna
print("Country:\n", df_country.isnull().mean().round(2))
print("City:\n", df_city.isnull().mean().round(2))
print("CountryLanguage:\n", df_lang.isnull().mean().round(2))

Country:
 Code              0.00
Name              0.00
Continent         0.00
Region            0.00
SurfaceArea       0.00
IndepYear         0.20
Population        0.00
LifeExpectancy    0.07
GNP               0.00
GNPOld            0.26
LocalName         0.00
GovernmentForm    0.00
HeadOfState       0.00
Capital           0.03
Code2             0.00
dtype: float64
City:
 ID             0.0
Name           0.0
CountryCode    0.0
District       0.0
Population     0.0
dtype: float64
CountryLanguage:
 CountryCode    0.0
Language       0.0
IsOfficial     0.0
Percentage     0.0
dtype: float64


In [46]:
# Comprobación de duplicados
print("Duplicados encontrados:")
print("country:", df_country.duplicated().sum())
print("city:", df_city.duplicated().sum())
print("countrylanguage:", df_lang.duplicated().sum())

Duplicados encontrados:
country: 0
city: 0
countrylanguage: 0


#### 2.4. Acciones correctivas

- No se eliminan nulos de campos como `HeadOfState`, `IndepYear`, `GNP` ya que pueden ser informativos.
- Se eliminan duplicados si existen (no deberían debido a las claves primarias en cada tabla).

In [41]:
df_country.drop_duplicates(inplace=True)
df_city.drop_duplicates(inplace=True)
df_lang.drop_duplicates(inplace=True)

print(df_country.duplicated().sum())  # Esperado: 0

0
